In [4]:
import pandas as pd
import plotly.express as px
import nbformat




In [5]:
Dinning = pd.read_excel("DinningData.xlsx")


In [7]:
Dinning.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   DEP            240 non-null    object 
 1   Semester       240 non-null    object 
 2   Month          240 non-null    object 
 3   Academic Year  240 non-null    object 
 4   Hours          180 non-null    float64
 5   Money Spent    240 non-null    float64
dtypes: float64(2), object(4)
memory usage: 11.4+ KB


In [9]:
missing_values = Dinning.isnull().sum()
missing_values


DEP               0
Semester          0
Month             0
Academic Year     0
Hours            60
Money Spent       0
dtype: int64

In [10]:
Dinning['Hours'].fillna(Dinning['Hours'].mean(), inplace=True)


/var/folders/b5/c2s8x6hx5bz57tz7p2pf4f2h0000gn/T/ipykernel_48244/2537690152.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Dinning['Hours'].fillna(Dinning['Hours'].mean(), inplace=True)


In [12]:
missing_values = Dinning.isnull().sum()
missing_values

DEP              0
Semester         0
Month            0
Academic Year    0
Hours            0
Money Spent      0
dtype: int64

In [13]:
Dinning

,DEP,Semester,Month,Academic Year,Hours,Money Spent
0,Res Dinning,Fall,Sep,20/21,3859.98,31844.8350
1,Res Dinning,Fall,Oct,20/21,4722.98,38964.5850
2,Res Dinning,Fall,Nov,20/21,2842.68,23452.1100
3,Res Dinning,Fall,Dec,20/21,1348.50,11125.1250
4,Res Dinning,J-Term,Jan,20/21,2247.99,18545.9175
...,...,...,...,...,...,...
235,Catering,Spring,Apr,23/24,119.09,1101.5825
236,Catering,Spring,May,23/24,58.38,540.0150
237,Catering,Summer,Jun,23/24,0.00,0.0000
238,Catering,Summer,July,23/24,0.00,0.0000
